In [2]:
from gpkit.shortcuts import Var, Model
from gpkit import units
import numpy as np

In [3]:
class electric_motor(Model):
    def setup(self):
        
        #from electric motor model found here: http://web.mit.edu/drela/Public/web/qprop/motorprop.pdf
        
        v = Var("v", "volts", "Voltage accross the motor")
        i = Var("i", "amps", "Current")
        i_0 = Var("i_0", 1, "amps", "Zero-load current")
        S = Var("S", 3, "-", "Number of Lipo-Battery Cells")
        Omega = Var("Omega", "rad/s", "Shaft rotational speed")
        K_v = Var("K_v", 115, "rad/s/volts", "Motor speed constant")
        R_m = Var("R_m",0.1, "ohms", "Motor internal resistance")
        C_T = Var("C_T", 0.002, "-", "Thrust Coefficient")
        C_P = Var("C_P", 1e-4, "-", "Power Coefficient")
        T = Var("T", "N", "Thrust")
        R = Var("R", 0.18, "m", "Propeller radius")
        V_tip = Var("V_{tip}", "m/s", "Tip velocity")
        V_tip_max = Var("V_{tip_{max}}", 100, "m/s", "Maximum tip velocity")
        rho = Var("\\rho", 1.225, "kg/m^3", "Air Density")
        i_max = Var("i_{max}",4,"amps","max current")
        
        constraints = [v == 3.7*S * units('volt'),
                       v >= Omega/K_v + i*R_m,
                       0.5*rho*Omega**2*R**4*np.pi*C_T >= T,
                       i >= 0.5*rho*Omega**2*R**5*np.pi*C_P*K_v + i_0,
                       #add weight as function of K_v, i, and T, 
                      ]
        cost = 1/T
        return cost, constraints
    
    def test(self):
        self.solve()

m = electric_motor()
sol = m.solve()
        

Using solver 'cvxopt'
Solving for 4 variables.
Solving took 0.0647 seconds.

Cost
----
 0.173 [1/N] 

Free Variables
--------------
Omega : 1196   [rad/s] Shaft rotational speed   
    T : 5.781  [N]     Thrust                   
    i : 6.983  [A]     Current                  
    v : 11.1   [V]     Voltage accross the motor

Constants
---------
 C_P : 0.0001            Power Coefficient           
 C_T : 0.002             Thrust Coefficient          
 K_v : 115     [rad/V/s] Motor speed constant        
   R : 0.18    [m]       Propeller radius            
 R_m : 0.1     [ohm]     Motor internal resistance   
   S : 3                 Number of Lipo-Battery Cells
\rho : 1.225   [kg/m**3] Air Density                 
 i_0 : 1       [A]       Zero-load current           

Sensitivities
-------------
 R_m : 0.1204  Motor internal resistance   
 C_P : 0.1032  Power Coefficient           
 i_0 : 0.01724 Zero-load current           
\rho : -0.8968 Air Density                 
 C_T : -1     